In [1]:
import pandas as pd
import numpy as ap
import lightgbm as lg
import re

In [2]:
DATA_PATH = r"../../../../../Data/Elo Merchant Category Recommendation/"

In [3]:
train = pd.read_csv(DATA_PATH + r"train.csv")
#(201917, 6)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'

In [4]:
test = pd.read_csv(DATA_PATH+r"test.csv")
#(123623, 5)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3'

In [5]:
merchants = pd.read_csv(DATA_PATH+r"merchants.csv")
#(334696, 22)
#'merchant_id', 'merchant_group_id', 'merchant_category_id', 'subsector_id', 'numerical_1',
#'numerical_2', 'category_1','most_recent_sales_range', 'most_recent_purchases_range',
#'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3','avg_sales_lag6', 
#'avg_purchases_lag6', 'active_months_lag6','avg_sales_lag12', 'avg_purchases_lag12',
#'active_months_lag12','category_4', 'city_id', 'state_id', 'category_2'

In [6]:
new_merchant = pd.read_csv(DATA_PATH+r"new_merchant_transactions.csv")
#(1963031, 14)
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'


In [7]:
hist = pd.read_csv(DATA_PATH+r"historical_transactions.csv")
#(29112361, 14)
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

**train,test**<br>
1. Convert 'first_active_month' column to "first_month","first_year","first_season"
+ Drop 'first_active_month'
+ Save to train_m.csv

In [8]:
test.loc[:,'first_active_month'].apply(type)

0         <class 'str'>
1         <class 'str'>
2         <class 'str'>
3         <class 'str'>
4         <class 'str'>
5         <class 'str'>
6         <class 'str'>
7         <class 'str'>
8         <class 'str'>
9         <class 'str'>
10        <class 'str'>
11        <class 'str'>
12        <class 'str'>
13        <class 'str'>
14        <class 'str'>
15        <class 'str'>
16        <class 'str'>
17        <class 'str'>
18        <class 'str'>
19        <class 'str'>
20        <class 'str'>
21        <class 'str'>
22        <class 'str'>
23        <class 'str'>
24        <class 'str'>
25        <class 'str'>
26        <class 'str'>
27        <class 'str'>
28        <class 'str'>
29        <class 'str'>
              ...      
123593    <class 'str'>
123594    <class 'str'>
123595    <class 'str'>
123596    <class 'str'>
123597    <class 'str'>
123598    <class 'str'>
123599    <class 'str'>
123600    <class 'str'>
123601    <class 'str'>
123602    <class 'str'>
123603    <class

In [9]:
train["first_month"] = train.loc[:,'first_active_month'].apply(lambda x: int(x.split("-")[1]))

In [11]:
test["first_month"] = test.loc[:,'first_active_month'].apply(lambda x: int(x.split("-")[1]))

AttributeError: 'float' object has no attribute 'split'

In [ ]:
train["first_year"] = train.loc[:,'first_active_month'].apply(lambda x: int(x.split("-")[0]))

In [ ]:
def ssn(x):
    x = int(x.split("-")[1])
    if x in [12,1,2]:
        return 0
    elif x<6:
        return 1
    elif x<9:
        return 2
    else:
        return 3

In [ ]:
train["first_season"] = train.loc[:,'first_active_month'].apply(ssn)

In [ ]:
train = train.drop(['first_active_month'],axis = 1)

In [ ]:
train.to_csv(DATA_PATH+r"train_m.scv")

Discard rows with NaN
Change "authorized_flag" 'Y' - > 10, 'N' - >1
Change "category_1"  'Y' - > 1, 'N' - >0
Change "category_3"  "A","B","C" - > 1,2,3
Change "purchase_date" to numerical
SUM log base = "city_id" (authorized_flag*category_1*installments*category_3*month_lag*purchase_amount*purchase_date*category_2*state_id*subsector_id)^merchant_category_id

In [ ]:
hist.loc[:,"subsector_id"].unique()

In [ ]:
new_merchant.head(10)

In [ ]:
test_m.shape

In [ ]:
train.loc[:,"first_active_month"] = train.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [ ]:
test.loc[:,"first_active_month"] = test.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [ ]:
train_m = pd.merge( train, hist, how = "inner", on = "card_id")
#(18030009, 19)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target'
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

In [ ]:
test_m = pd.merge( test, hist, how = "inner", on = "card_id")
#(11082352, 18)
#'first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
#'authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
#'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
#'purchase_amount', 'purchase_date', 'category_2', 'state_id','subsector_id'

In [ ]:
train_m.to_csv(DATA_PATH+r"train_m.csv")

In [ ]:
test_m.to_csv(DATA_PATH+r"test_m.csv")

In [ ]:
part = np.random.rand(len(df_train))<0.7

In [ ]:
valid_m = df_train[~part]